In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
 

# HYPERPARAMETERS
num_classes = 7
lr = 0.01
num_epochs = 60
batch_size = 32
weight_decay = 0.005
momentum = 0.9

# Parameters for plotting
test_losses = []
train_losses = []
test_correct = []
train_correct = []

In [ ]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.5551611804640287, 0.491291223366287, 0.44530976833072267],
            std=[0.2700789332643185, 0.254130887428006, 0.25608458818420854],
        )
])


# Load the custom dataset
train_dataset = datasets.ImageFolder(root='dataset/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = datasets.ImageFolder(root='dataset/valid', transform=transform)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

test_dataset = datasets.ImageFolder(root='dataset/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Load the pre-trained RegNet model
model = models.regnet_y_400mf(weights=models.RegNet_Y_400MF_Weights.DEFAULT)
 
# Modify the last fully connected layer for the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
 
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)

In [7]:
# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
 
for epoch in range(num_epochs):
    trn_corr = 0
    epoch_losses = []  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
 
        optimizer.zero_grad()
 
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')
    
    # (NERI) TO TRACK THE LOSS AND CORRECT OF THE CNN
    epoch_losses.append(loss.item())  # Append every batch
    train_losses.append(sum(epoch_losses) / len(epoch_losses))  # Store epoch average
    train_correct.append(trn_corr)

    # Validating the model 
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Accuracy of the network on the test images: {100 * correct / total}%')

Epoch 1, Loss: 0.9850288703070058
Accuracy of the network on the test images: 30.238095238095237%


KeyboardInterrupt: 

In [ ]:
# Test the model
with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # For plotting
            loss = criterion(outputs, labels)
            test_losses.append(loss.item())
            test_correct.append(correct)
            
            del images, labels, outputs
    
        print('Accuracy of the network on the {} test images: {} %'.format(3500, 100 * correct / total))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# GRAPH THE LOSS AT EPOCHS
# convert tensors to floats
train_loss_vals = [l.item() if isinstance(l, torch.Tensor) else float(l) for l in train_losses]
test_loss_vals  = [l.item() if isinstance(l, torch.Tensor) else float(l) for l in test_losses]

# Plot directly - already per-epoch
plt.plot(range(len(train_loss_vals)), train_loss_vals, label='Training Loss')
plt.plot(range(len(test_loss_vals)),  test_loss_vals,  label='Test Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# GRAPH ACCURACY PER EPOCH
# Convert tensors to CPU and then to a list of numbers for plotting
train_correct_cpu = [t.cpu().item() if isinstance(t, torch.Tensor) else t for t in train_correct]
test_correct_cpu = [t.cpu().item() if isinstance(t, torch.Tensor) else t for t in test_correct]

plt.plot([t/2450 for t in train_correct_cpu], label='Training Accuracy') # Use the CPU version for plotting
plt.plot([t/420 for t in test_correct_cpu], label = 'Validation Accuracy') # Use the CPU version for plotting
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show() 

In [ ]:
torch.save(model.state_dict(), 'Regnet_model(State_dict).pth')
torch.save(model, 'Regnet_model(Full model).pth')